# Prepare Development Environment

This will create the virtual environment

In [ ]:
!conda create -c pytorch -c nvidia -n gen_voice pytorch torchvision torchaudio pytorch-cuda=11.8 -y
!conda activate gen_voice
!conda install conda-forge::ffmpeg
!pip install -r requirements.txt
!playwright install

If ffmpeg doesn't work correctly on Linux the following command may help

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

# Import Libraries

In [1]:
from financial_chatbot import ChatBot
import speech_recognition as sr
import gradio as gr

%load_ext gradio

c:\Users\holes\.conda\envs\ik\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pull Data for LLM Context

In [12]:
!python extract_web_data.py

# Check for Microphones

In [ ]:
# List the available microphones
sr.Microphone.list_microphone_names()

## Full Audio
If you're system has speakers and a microphone you can run in full audio using the communication loop below

In [10]:
# Initialize the chatbot specifying the index of the mic if one exists and whether or not speakers will be used.
# For best results a headset with a mic is recommended.
chatbot = ChatBot(mic_id=2, enable_speakers=False)

history = []
while True:
    guess = chatbot.recognize_speech_from_mic()
    if guess["success"]:
        prompt = guess['transcription']
        print(f'Prompt: {prompt} \n')

        if 'goodbye' in prompt:
            chatbot.communicate('See you next time')
            break
        
        response = chatbot.respond(prompt, history)
        history.append([guess['transcription'], response])
        print(f'{response}\n\n')
    else:
        chatbot.communicate(f"Error: {guess["error"]}")

Prompt: what are key indicators 

Key indicators in financial markets typically include:

1. **Interest Rates**: Set by central banks, interest rates influence borrowing costs and economic activity. Higher rates can slow economic growth, while lower rates can stimulate it.

2. **Inflation Rates**: Measures the rate at which the general level of prices for goods and services is rising. Central banks monitor inflation to decide on monetary policy.

3. **Gross Domestic Product (GDP)**: Represents the total dollar value of all goods and services produced over a specific time period. It is used to gauge the health of a country's economy.

4. **Unemployment Rate**: Indicates the percentage of the total labor force that is unemployed and actively seeking employment. It is a key indicator of economic health.

5. **Consumer Price Index (CPI)**: Measures the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services.

6. **Producer Price Ind

## Speakers Optional (no mic)
The following gradio UI will allow you to communicate with the LLM.

In [3]:
%%blocks

# Create a gradio chat interface wire it to our chatbot
chatbot = ChatBot(enable_speakers=True, threaded=True)
with gr.Blocks() as demo:
    gr.ChatInterface(chatbot.respond)